
# <span style="font-family:Ink Free"><span style="color:DarkRed">**~~Manual~~**</span></span>    <span style="color:green"><span style="font-family:Impact">_AUTOMATED!_</span></span>  <span style="color:LightBlue"></span>   
# &emsp; &emsp; &emsp; &emsp;  **Creation** of 
# &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; <span style="color:LightBlue"><span style="font-family:Impact">I</span><span style="font-family:papyrus">N</span><span style="font-family:Arial">T</span><span style="font-family:Bahnschrift">e</span><span style="font-family:Avanta Garde">R</span><span style="color:DarkOrange"><span style="font-family:Brush Script MT">A</span><span style="font-family:Courier">C</span><span style="font-family:Lucida Bright">T</span><span style="font-family:MV Boli">i</span><span style="font-family:Avanta Garde">V</span><span style="font-family:Helvetica">e</span></span></span>
 # &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; **Web Maps**
 ---

## This will create a <span style="color:SpringGreen">___Folium___</span> webmap for every matching pair of watershed and stream parquet files in the specified input filepath locations.
---
### The <span style="color:CornflowerBlue">___create_webmap___</span> function creates the folium/leaflet webmap, adds widgets and plugins, and contains the styling options. The plugin selection and styling of the webmap are placeholders for demonstration purposes. 


In [ ]:
import os
import re
import geopandas as gpd
import folium
import folium.plugins
from folium.plugins import Fullscreen, MeasureControl, MousePosition, MiniMap, Search

In [ ]:

# This function matches watershed and stream parquet files based
# on the 8 digit hydrologic unit code in the file names. It is
# used in the process_parquet_files function.
def extract_common_number(filename):
    match = re.search(r'\d{8}', filename)
    return match.group(0) if match else None

# This function adds and styles a banner to the html with base text
# and inserts file specific name information for each watershed.
def add_banner_to_html(output_html, title):
    banner_html = f"""
    <div id="banner">
        <h3>{title}</h3>
         <p>Map showing a Texas watershed and drainage network.</p>
    </div>
    <style>
        body {{
            margin: 0;
            padding: 0;
        }}    
        #banner {{
            position: fixed;
            top: 0;
            width: 100%;
            background-color: #333;
            color: white;
            text-align: center;
            padding: 10px;
            z-index: 800;
        }}
        .folium-map {{
            position: absolute;
            top: 100px;
            z-index: 500;
        }}
        .leaflet-top, .leaflet-control {{
            margin-top: 110px;
            z-index: 700;
        }}
    </style>
    """
    with open(output_html, 'r') as file:
        content = file.read()
    content = content.replace('<body>', f'<body>{banner_html}', 1)
    with open(output_html, 'w') as file:
        file.write(content)

# This function will Initialize/Create a new folium map centered on Texas
# It uses the add_banner_to_html function.
def create_webmap(gdf_ws, gdf_strm, output_html, title):
    m = folium.Map(location=[31.9686, -99.9018], zoom_start=6, control_scale=True, font_size='1rem')
    
    # Add an optional locator map.
    MiniMap(position="bottomleft", 
            width=175, 
            height=125, 
            zoom_level_offset=-5,
            toggle_display=True,
            auto_toggle_display=True).add_to(m)
    
    # Configure what watershed data will be shown in the popup.
    popupws = folium.GeoJsonPopup(
        fields=["name", "huc8", "states", "areasqkm"],
        aliases=["Watershed", "HU8 code", "State(s)", "Area sq/km"],
        localize=True,
        labels=True,
        style="background-color: green;",
    )

    # Configure what data will be shown on mouse over.
    tooltipws = folium.GeoJsonTooltip(
        fields=["name", "huc8"],
        aliases=["Watershed", "HU8 code"],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 5px;
            box-shadow: 3px;
        """,
        max_width=800,
    )
    
    # Configure what stream data will be shown in the popup.
    popupstr = folium.GeoJsonPopup(
        fields=["gnis_name", "hu8", "reachcode", "lengthkm"],
        aliases=["Name", "HU8 code", "Reach code", "Length km"],
        localize=True,
        labels=True,
        style="background-color: green;",
    )

    # Configure what data will be shown on mouse over.
    tooltipstr = folium.GeoJsonTooltip(
        fields=["gnis_name", "hu8"],
        aliases=["Name", "HU8 code"],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 5px;
            box-shadow: 3px;
        """,
        max_width=800,
    )
    
    
    # Add stream data to the map
    streamlayer = folium.GeoJson(
        gdf_strm,
        name='Streams',
        control=True,
        labels=True,
        overlay=True,
        zoom_on_click=True,
        style_function=lambda x: {
            'color': '#3341FF' if x['properties']['gnis_name'] is not None else '#33FFCF',
            'weight': 5 if x['properties']['gnis_name'] is not None else 3,
        },
        tooltip=tooltipstr,
        popup=popupstr,
    ).add_to(m)
    
    
    # Add the  watershed data to the map
    wslayer = folium.GeoJson(
        gdf_ws,
        name='Watershed',
        labels=True,
        overlay=True,
        popup_keep_highlighted= True,
        zoom_on_click=True,
        style_function=lambda x: {
            'color': 'orange',
            'fillColor': 'orange',
            'opacity': 0.5,
            'weight': 1.5
        },
        tooltip=tooltipws,
        popup=popupws,
    ).add_to(m)
    
    
    # Add a stream name search box for stream layer data.
    streams_search = Search(
        layer=streamlayer,
        geom_type="Polyline",
        placeholder="Search for Streams by Name",
        collapsed=False,
        search_label="gnis_name",
        search_zoom=13
    ).add_to(m)
    
    # Add a reachcode search box for stream layer data.
    search_reachcode = Search(
        layer=streamlayer,
        geom_type="Polyline",
        placeholder="Search for Streams by Reachcode",
        collapsed=False,
        search_label="reachcode",
        search_zoom=13
    ).add_to(m)
    
    
    # Create and style a legend for the stream layer.
    legend_html = '''
         <div style="position: fixed; 
         top: 320px; right: 10px; width: 180px; height: 50px; 
         background-color: white; z-index:9999; font-size:12px;
         border:2px solid gray; padding: 10px; font-family: sans-serif;
         text-transform: capitalize;">
         <i class="fa fa-map-marker fa-1x" style="color:#3341FF"></i>&nbsp; Named Streams<br>
         <i class="fa fa-map-marker fa-1x" style="color:#33FFCF"></i>&nbsp; Unnamed Streams
         </div>
         '''

    # This adds the legend to the map.
    m.get_root().html.add_child(folium.Element(legend_html))
    
    
    # Create styling for the two search boxes.
    search_css = '''
    <style>
    .leaflet-control-search {
        position: fixed;
        top: 150px;
        left: 60px;
        z-index: 9999;
    }
    .leaflet-control-search:nth-of-type(2) {
        position: fixed;
        top: 150px;
        left: 60px;
        z-index: 9999;
    }
    </style>
    '''
    
    # Use the style in the map.
    m.get_root().html.add_child(folium.Element(search_css))

    # Fits the map to the overlays when it loads.
    folium.FitOverlays(padding=20 ,fit_on_map_load=True)

    # Add fullscreen button
    Fullscreen().add_to(m)

    # Add a Measurement tool for length and area calculations.
    MeasureControl().add_to(m)
    
    # This will show the Latitude and Longitude for the mouse pointer position.
    MousePosition(
    position="bottomright",
    prefix="Lat | Long:",
    separator=" | ",
    ).add_to(m)
    
    # Add layer control to toggle visibility for the layers.
    folium.LayerControl(
        draggable=False 
    ).add_to(m)


    # Save the map as an HTML file.
    m.save(output_html)
    
    # Call the function to add the banner.
    add_banner_to_html(output_html, title)
    

# This function uses the extract_common_number and create_webmap functions
# to automatically create webmaps for all watershed and streams parquet files
# in the specified file path locations unless a watershed file does not have a
# matching stream file.
def process_parquet_files(folder_path_ws, folder_path_strm, output_folder):
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Get the list of watershed and stream files
    ws_files = [f for f in os.listdir(folder_path_ws) if f.endswith('.parquet')]
    strm_files = [f for f in os.listdir(folder_path_strm) if f.endswith('.parquet')]

    for ws_filename in ws_files:
        ws_file_path = os.path.join(folder_path_ws, ws_filename)
        ws_common_number = extract_common_number(ws_filename)
        output_html = os.path.join(output_folder, ws_filename.replace('.parquet', '.html'))

        # Read watershed parquet file
        gdf_ws = gpd.read_parquet(ws_file_path)
        gdf_ws = gdf_ws.to_crs(epsg=4326)
        
        # Get the watershed name from the 'name' column in the watershed parquet file.
        watershed_name = gdf_ws['name'].iloc[0]

        # Find the corresponding stream file
        strm_filename = next((f for f in strm_files if ws_common_number in f), None)
        if strm_filename:
            strm_file_path = os.path.join(folder_path_strm, strm_filename)

            # Read stream parquet file
            gdf_strm = gpd.read_parquet(strm_file_path)
            gdf_strm = gdf_strm.to_crs(epsg=4326)
        
            # Create and save the webmap
            title = f"Map for the {watershed_name} Watershed - HU8 Code: {ws_common_number}"
            create_webmap(gdf_ws, gdf_strm, output_html, title)
        else:
            print(f"No matching stream file found for {ws_filename}")

# Specify input and output folder paths
# This is the location of the watershed parquet files.
folder_path_ws = r"C:\GIS\adatawm\ws"
#
# This is the location of the stream parquet files.
folder_path_strm = r"C:\GIS\adatawm\strms"
#
# This is the location where the completed webmap html files
# will be saved.
output_folder = r"C:\GIS\adatawm\out_wmaps"

# Process all parquet files in the input folders
process_parquet_files(folder_path_ws, folder_path_strm, output_folder)
